In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import pandas as pd
import numpy as np
sns.set()
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

In [3]:
cols = ['age','sex','chestpain','restingbp','cholestrol','fastbloodsugar','restecg','thalach','exang','oldpeak','slope','ca','thal','num']
df_cleveland = pd.read_table('./Heart_Disease_Dataset/processed.cleveland.data',names=cols,sep=',')
df_va = pd.read_table('./Heart_Disease_Dataset/processed.va.data',names=cols,sep=',')
df_switzerland = pd.read_table('./Heart_Disease_Dataset/processed.switzerland.data',names=cols,sep=',')
df_hungarian = pd.read_table('./Heart_Disease_Dataset/processed.hungarian.data',names=cols,sep=',')

In [4]:
result = pd.concat([df_cleveland,df_hungarian,df_switzerland,df_va])

In [5]:
describe = result.describe()
my_index = list(range(0,920))


In [6]:
data=result
data.reset_index(inplace=True)

In [75]:
data.replace('?',np.nan,inplace=True)
data.isnull().sum()

age               0
sex               0
chestpain         0
restingbp         0
cholestrol        0
fastbloodsugar    0
restecg           0
thalach           0
exang             0
oldpeak           0
num               0
dtype: int64

In [8]:
data.drop(['slope','ca','thal','index'],1,inplace=True)

In [20]:
data.dropna(inplace=True)
pd.to_numeric(data['restingbp'],errors='coerce')
data.convert_objects(convert_numeric=True)
#data['restingbp']=data['restingbp'].astype(float)
data = data.astype(float)

In [24]:
X = data.values[:,:10]
y = data.values[:,10]

In [40]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=4)
k_neighbor = KNeighborsClassifier()

In [33]:
neighbor_opt = list(range(1,10))
weights_opt = ['uniform','distance']
algo_opt = ['auto','kd_tree','ball_tree','brute']
param_grid = dict(n_neighbors=neighbor_opt,weights=weights_opt,algorithm=algo_opt)
grid = GridSearchCV(k_neighbor,param_grid,cv=10,scoring='accuracy',n_jobs=-1)

In [34]:
%time grid.fit(X_train,y_train)
grid.best_params_

CPU times: user 858 ms, sys: 127 ms, total: 985 ms
Wall time: 1.7 s


{'algorithm': 'auto', 'n_neighbors': 6, 'weights': 'uniform'}

In [47]:
grid.best_score_

0.51711711711711716

In [46]:
y_pred = grid.predict(X_test)
print(accuracy_score(y_pred,y_test))
confusion_matrix(y_test,y_pred)


0.432432432432


array([[60, 16,  2,  2,  0],
       [32, 15,  5,  5,  0],
       [ 8,  6,  4,  1,  0],
       [12,  8,  0,  1,  0],
       [ 2,  5,  0,  1,  0]])

In [66]:
from sklearn.svm import SVC
gama=np.linspace(0.0,0.2,20)
c = np.linspace(0.5,1.5,10)
svm_model = SVC(gamma=0.01)
param_grid_new = dict(C=c,gamma=gama)
grid_new = GridSearchCV(svm_model,param_grid=param_grid_new,cv=10,scoring='accuracy',n_jobs=-1)
grid_new.fit(X_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.01,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=-1,
       param_grid={'gamma': array([ 0.     ,  0.01053,  0.02105,  0.03158,  0.04211,  0.05263,
        0.06316,  0.07368,  0.08421,  0.09474,  0.10526,  0.11579,
        0.12632,  0.13684,  0.14737,  0.15789,  0.16842,  0.17895,
        0.18947,  0.2    ]), 'C': array([ 0.5    ,  0.61111,  0.72222,  0.83333,  0.94444,  1.05556,
        1.16667,  1.27778,  1.38889,  1.5    ])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='accuracy', verbose=0)

In [67]:
y_pred_svm = grid_new.predict(X_test)
grid_new.best_score_
print(accuracy_score(y_pred,y_test))
print(confusion_matrix(y_test,y_pred))

0.432432432432
[[60 16  2  2  0]
 [32 15  5  5  0]
 [ 8  6  4  1  0]
 [12  8  0  1  0]
 [ 2  5  0  1  0]]


In [70]:
from sklearn.ensemble import RandomForestClassifier
forest_grid=RandomForestClassifier(n_jobs=-1)
criterion_list = ['gini','entropy']
n_estimators_list = list(range(5,15))
max_features_list = ['auto','log2',None]
param_grid_forest = dict(n_estimators=n_estimators_list,max_features=max_features_list,criterion=criterion_list)
grid_forest = GridSearchCV(forest_grid,param_grid_forest,cv=10,scoring='accuracy',n_jobs=-1)

In [71]:
%time grid_forest.fit(X_train,y_train)

CPU times: user 1.1 s, sys: 133 ms, total: 1.23 s
Wall time: 32.3 s


GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'max_features': ['auto', 'log2', None], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='accuracy', verbose=0)

In [74]:
y_pred_foest = grid_forest.predict(X_test)
print(grid_forest.best_score_,grid_forest.best_params_)

0.605405405405 {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 8}


In [73]:
print(accuracy_score(y_pred_foest,y_test))

0.486486486486
